In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!gdown --id 1F9XcsAkQ4fsN8IRVxRXfUwTtwqc5bd3N #X_test_embed
!gdown --id 1XCnEDzTXGTXb4NrDYimZoBmyyf1w398w #X_train_embed
!gdown --id 1jMYFF6px1ujxp7O13zMGdMLt-t7S7mjA #Y_test_embed
!gdown --id 14_qieFo-rp9DDzoa6ESpMwwNx4VgsenY #Y_train_embed

In [ ]:
!gdown --id 1pJL2m0-7XwzkjgWLxA1gcrX-lPJPHvNi #X_test7
!gdown --id 10uFQmiohMvXznxefaGU-YP0Kut6FHmtZ #X_train7
!gdown --id 1TLGdPkJPKsXBtjlInIWcDAtWz3WWbCy6 #Y_test7
!gdown --id 1XlvYDH_58T001jS89N7uUWboRC8Bw86o #Y_train7

In [ ]:
!gdown --id 1C-YUtRF80X6SpRqDZcENeBLYE8wnUCzD # Char Tokenizer

In [ ]:
!gdown --id 1WuNvc0Hwi5OHnUhfuB9aVuoSvHgmg5OH #CNN-Hybrid model

In [ ]:
import numpy as np
import pandas as pd
import joblib
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.utils.class_weight import compute_class_weight
from urllib.parse import urlparse
import re

In [ ]:
X_train_w2v = joblib.load("/kaggle/working/X_train_embed.pkl")
X_test_w2v = joblib.load("/kaggle/working/X_test_embed.pkl")
Y_train_w2v = joblib.load("/kaggle/working/Y_train_embed.pkl")
Y_test_w2v = joblib.load("/kaggle/working/Y_test_embed.pkl")
X_train_c2v = joblib.load("/kaggle/working/X_train7.pkl")
X_test_c2v = joblib.load("/kaggle/working/X_test7.pkl")
Y_train_c2v = joblib.load("/kaggle/working/Y_train7.pkl")
Y_test_c2v = joblib.load("/kaggle/working/Y_test7.pkl")

tokenizer = joblib.load("/kaggle/working/PhiUSIIL_Char_Tokenizer.joblib")
vocab_size = len(tokenizer.word_index) + 1
model = load_model("/kaggle/working/PhiUSIIL_CNN_Hybrid.keras")

print(f"\n✅ Data loaded successfully!")
print(f"   📊 Word2Vec embeddings: Train={X_train_w2v.shape}, Test={X_test_w2v.shape}")
print(f"   📊 Char sequences: Train={X_train_c2v.shape}, Test={X_test_c2v.shape}")
print(f"   📊 Labels: Train={Y_train_w2v.shape}, Test={Y_test_w2v.shape}")
print(f"   📚 Char vocabulary size: {vocab_size}")
print(f"   📊 Class distribution: Benign={np.sum(Y_train_w2v == 1):,}, Malicious={np.sum(Y_train_w2v == 0):,}")

classes = np.unique(Y_train_w2v)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=Y_train_w2v)
class_weights = dict(zip(classes, weights))
print(f"   📊 Class weight: {class_weights}")

In [ ]:
def PhiUSIIL_CNN_Hybrid(word_embedding_dim=50, char_embedding_dim=32, char_max_len=200, vocab_size=106,
        num_filters=256, filter_sizes=[3, 4, 5, 6], dropout_rate=0.5, fc_units=[512, 256, 128]):
    # ========================================== Character-Level =======================================================
    char_input = layers.Input(shape=(char_max_len,), name='char_input')
    # Char Embedding Layer
    char_embedding = layers.Embedding(input_dim=vocab_size, output_dim=char_embedding_dim,
                                     input_length=char_max_len, name='char_embedding',
                                     embeddings_regularizer=keras.regularizers.l2(1e-6))(char_input)
    # Parallel Convolutional Filters
    char_conv_blocks = []
    for filter_size in filter_sizes:
        conv = layers.Conv1D(filters=num_filters, kernel_size=filter_size, activation='relu',
                             padding='same', kernel_regularizer=keras.regularizers.l2(1e-6),
                             name=f'char_conv_{filter_size}')(char_embedding)
        # Batch Normalization
        conv = layers.BatchNormalization(name=f'char_bn_{filter_size}')(conv)
        # Global Max Pooling
        pool = layers.GlobalMaxPooling1D(name=f'char_pool_{filter_size}')(conv)
        char_conv_blocks.append(pool)
    # Concatenate all pooled features
    char_features = layers.Concatenate(name='char_concat')(char_conv_blocks)
    char_features = layers.Dropout(dropout_rate, name='char_dropout')(char_features)
    char_features = layers.Dense(fc_units[0], activation='relu',
                                 kernel_regularizer=keras.regularizers.l2(1e-6), name='char_fc')(char_features)
    char_features = layers.BatchNormalization(name='char_fc_bn')(char_features)
    # ========================================== Word-Level ==============================================================
    word_input = layers.Input(shape=(word_embedding_dim,), name='word_input')
    # Dense layers
    word_features = layers.Dense(512, activation='relu',
                                 kernel_regularizer=keras.regularizers.l2(1e-6), name='word_fc1')(word_input)
    word_features = layers.BatchNormalization(name='word_bn1')(word_features)
    word_features = layers.Dropout(dropout_rate, name='word_dropout1')(word_features)

    word_features = layers.Dense(fc_units[0], activation='relu',
                                 kernel_regularizer=keras.regularizers.l2(1e-6), name='word_fc2')(word_features)
    word_features = layers.BatchNormalization(name='word_bn2')(word_features)
    word_features = layers.Dropout(dropout_rate, name='word_dropout2')(word_features)
    # ======================================== Concatenate Char & Word ===================================================
    combined = layers.Concatenate(name='combined_features')([char_features, word_features])
    # ========================================== Classification Head =====================================================
    x = combined
    for i, units in enumerate (fc_units[1:], 1):
        x = layers.Dense(units, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-2), 
                        name=f'fc_{i}')(x)
        x = layers.BatchNormalization(name=f'fc_bn_{i}')(x)
        x = layers.Dropout(dropout_rate, name=f'dropout_{i}')(x)
    # ============================================ Output layer ==========================================================
    output = layers.Dense(1, activation='sigmoid', name='output')(x)
    # ============================================= Build Model ==========================================================
    model = Model(inputs=[char_input, word_input], outputs=output, name='PhiUSIIL_CNN_Hybrid')

    return model

In [ ]:
print("\n🏗️ Building PhiUSIIL_CNN_Hybrid model...")
model = PhiUSIIL_CNN_Hybrid(
    word_embedding_dim=50, char_embedding_dim=32, char_max_len=200, vocab_size=vocab_size,
    num_filters=256, filter_sizes=[3, 4, 5, 6], dropout_rate=0.5, fc_units=[512, 256, 128] )

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.001, clipnorm=1.0),
    loss='binary_crossentropy',
    metrics=[
        'accuracy',
        keras.metrics.AUC(name='auc'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall')
    ]
)

In [ ]:
print("\n📊 Model Architecture:")
model.summary()
total_params = model.count_params()
print(f"\n💡 Total Parameters: {total_params:,}")

In [ ]:
callbacks = [
    ModelCheckpoint(
        filepath='/kaggle/working/PhiUSIIL_CNN_Hybrid_Checkpoints.keras',
        save_best_only=True,
        verbose=1
    ),
    EarlyStopping(
        monitor='val_loss',
        mode='max',
        patience=7,
        verbose=1,
        restore_best_weights=True
    ),
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        verbose=1,
        min_lr=1e-7
    )
]

In [ ]:
print("\n" + "=" * 80)
print("🚀 TRAINING PhiUSIIL-CNN-Hybrid Model")
print("=" * 80)

result = model.fit(
    [X_train_c2v, X_train_w2v],
    Y_train_w2v,
    batch_size=512,
    epochs=100,
    validation_split=0.15,
    callbacks=callbacks,
    class_weight=class_weights,
    verbose=1
)

In [ ]:
print("\n" + "=" * 80)
print("📈 EVALUATING MODEL ON TEST SET")
print("=" * 80)

test_results = model.evaluate(
    [X_test_c2v, X_test_w2v],
    Y_test_w2v,
    batch_size=512,
    verbose=1
)

print(f"\n✅ Test Results:")
print(f"   - Loss: {test_results[0]:.4f}")
print(f"   - Accuracy: {test_results[1]:.4f}")
print(f"   - AUC: {test_results[2]:.4f}")
print(f"   - Precision: {test_results[3]:.4f}")
print(f"   - Recall: {test_results[4]:.4f}")

In [ ]:
print("\n🔮 Generating predictions...")
y_pred_proba = model.predict([X_test_c2v, X_test_w2v], batch_size=512, verbose=1)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, roc_curve

print("\n" + "=" * 80)
print("📊 DETAILED PERFORMANCE METRICS")
print("=" * 80)

print("\n📋 Classification Report:")
print(classification_report(
    Y_test_w2v, 
    y_pred, 
    target_names=['Phishing (0)', 'Benign (1)'],
    digits=4
))

print("\n🔢 Confusion Matrix:")
cm = confusion_matrix(Y_test_w2v, y_pred)
print(cm)
print(f"\nTrue Negatives (TN): {cm[0,0]:,}")
print(f"False Positives (FP): {cm[0,1]:,}")
print(f"False Negatives (FN): {cm[1,0]:,}")
print(f"True Positives (TP): {cm[1,1]:,}")

# Calculate additional metrics
fpr_rate = cm[0,1] / (cm[0,0] + cm[0,1])
fnr_rate = cm[1,0] / (cm[1,0] + cm[1,1])
print(f"\nFalse Positive Rate: {fpr_rate:.4f}")
print(f"False Negative Rate: {fnr_rate:.4f}")

# ROC-AUC
roc_auc = roc_auc_score(Y_test_w2v, y_pred_proba)
print(f"\n🎯 ROC-AUC Score: {roc_auc:.4f}")

# ==================== VISUALIZATIONS ====================
print("\n📊 Creating visualizations...")

fig = plt.figure(figsize=(20, 12))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# 1. Training History - Loss
ax1 = fig.add_subplot(gs[0, 0])
ax1.plot(result.history['loss'], label='Train Loss', linewidth=2)
ax1.plot(result.history['val_loss'], label='Val Loss', linewidth=2)
ax1.set_title('Model Loss', fontsize=14, fontweight='bold')
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Loss')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Training History - Accuracy
ax2 = fig.add_subplot(gs[0, 1])
ax2.plot(result.history['accuracy'], label='Train Accuracy', linewidth=2)
ax2.plot(result.history['val_accuracy'], label='Val Accuracy', linewidth=2)
ax2.set_title('Model Accuracy', fontsize=14, fontweight='bold')
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Accuracy')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Training History - AUC
ax3 = fig.add_subplot(gs[0, 2])
ax3.plot(result.history['auc'], label='Train AUC', linewidth=2)
ax3.plot(result.history['val_auc'], label='Val AUC', linewidth=2)
ax3.set_title('Model AUC', fontsize=14, fontweight='bold')
ax3.set_xlabel('Epoch')
ax3.set_ylabel('AUC')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Confusion Matrix
ax4 = fig.add_subplot(gs[1, 0])
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax4, 
            xticklabels=['Phishing', 'Benign'],
            yticklabels=['Phishing', 'Benign'])
ax4.set_title('Confusion Matrix', fontsize=14, fontweight='bold')
ax4.set_ylabel('True Label')
ax4.set_xlabel('Predicted Label')

# 5. ROC Curve
ax5 = fig.add_subplot(gs[1, 1])
fpr, tpr, thresholds = roc_curve(Y_test_w2v, y_pred_proba)
ax5.plot(fpr, tpr, linewidth=2, label=f'ROC Curve (AUC = {roc_auc:.4f})')
ax5.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
ax5.set_title('ROC Curve', fontsize=14, fontweight='bold')
ax5.set_xlabel('False Positive Rate')
ax5.set_ylabel('True Positive Rate')
ax5.legend()
ax5.grid(True, alpha=0.3)

# 6. Prediction Distribution
ax6 = fig.add_subplot(gs[1, 2])
ax6.hist(y_pred_proba[Y_test_w2v == 0], bins=50, alpha=0.6, label='Malicious', color='red')
ax6.hist(y_pred_proba[Y_test_w2v == 1], bins=50, alpha=0.6, label='Benign', color='green')
ax6.axvline(0.5, color='black', linestyle='--', linewidth=2, label='Threshold')
ax6.set_title('Prediction Distribution', fontsize=14, fontweight='bold')
ax6.set_xlabel('Predicted Probability')
ax6.set_ylabel('Frequency')
ax6.legend()
ax6.grid(True, alpha=0.3)

# 7. Precision-Recall
ax7 = fig.add_subplot(gs[2, 0])
ax7.plot(result.history['precision'], label='Train Precision', linewidth=2)
ax7.plot(result.history['val_precision'], label='Val Precision', linewidth=2)
ax7.set_title('Precision', fontsize=14, fontweight='bold')
ax7.set_xlabel('Epoch')
ax7.set_ylabel('Precision')
ax7.legend()
ax7.grid(True, alpha=0.3)

# 8. Recall
ax8 = fig.add_subplot(gs[2, 1])
ax8.plot(result.history['recall'], label='Train Recall', linewidth=2)
ax8.plot(result.history['val_recall'], label='Val Recall', linewidth=2)
ax8.set_title('Recall', fontsize=14, fontweight='bold')
ax8.set_xlabel('Epoch')
ax8.set_ylabel('Recall')
ax8.legend()
ax8.grid(True, alpha=0.3)

# 9. Metrics Summary Table
ax9 = fig.add_subplot(gs[2, 2])
ax9.axis('tight')
ax9.axis('off')

metrics_data = [
    ['Metric', 'Value'],
    ['Accuracy', f'{test_results[1]:.4f}'],
    ['AUC', f'{test_results[2]:.4f}'],
    ['Precision', f'{test_results[3]:.4f}'],
    ['Recall', f'{test_results[4]:.4f}'],
    ['F1-Score', f'{2 * test_results[3] * test_results[4] / (test_results[3] + test_results[4]):.4f}'],
    ['FPR', f'{fpr_rate:.4f}'],
    ['FNR', f'{fnr_rate:.4f}']
]

table = ax9.table(cellText=metrics_data, cellLoc='center', loc='center',
                  colWidths=[0.4, 0.3])
table.auto_set_font_size(False)
table.set_fontsize(11)
table.scale(1, 2.5)

for i in range(len(metrics_data)):
    if i == 0:
        table[(i, 0)].set_facecolor('#4CAF50')
        table[(i, 1)].set_facecolor('#4CAF50')
        table[(i, 0)].set_text_props(weight='bold', color='white')
        table[(i, 1)].set_text_props(weight='bold', color='white')
    else:
        table[(i, 0)].set_facecolor('#E8F5E9')
        table[(i, 1)].set_facecolor('#E8F5E9')

plt.suptitle('PhiUSIIL_CNN_Hybrid - Complete Performance Analysis', 
             fontsize=16, fontweight='bold', y=0.995)
plt.savefig('/kaggle/working/PhiUSIIL_CNN_Hybrid_results.png', 
            dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
rng = np.random.RandomState(42)

def stratified_sample_indices(y, n_per_class, random_state=None):
    """Return indices with exactly n_per_class for each unique class in y.
       Throws if class sizes < n_per_class.
    """
    random_state = random_state or np.random
    classes = np.unique(y)
    idxs = []
    for c in classes:
        pos = np.where(y == c)[0]
        if len(pos) < n_per_class:
            raise ValueError(f"Not enough samples for class {c}: have {len(pos)}, need {n_per_class}")
        chosen = random_state.choice(pos, size=n_per_class, replace=False)
        idxs.append(chosen)
    return np.concatenate(idxs)

In [ ]:
bg_per_class = 1000   # 2000 total
test_per_class = 5000 # 10000 total
Y = np.asarray(Y_train_w2v)

idx_bg = stratified_sample_indices(Y, n_per_class=bg_per_class, random_state=rng)
# remove used indices from pool for test to avoid overlap
remaining_idx = np.setdiff1d(np.arange(len(Y)), idx_bg)
Y_remaining = Y[remaining_idx]

# sample test from remaining in balanced way
def stratified_from_pool(pool_idx, pool_y, n_per_class, random_state=None):
    random_state = random_state or np.random
    classes = np.unique(pool_y)
    idxs = []
    for c in classes:
        pos_in_pool = pool_idx[pool_y == c]
        if len(pos_in_pool) < n_per_class:
            raise ValueError(f"Not enough samples in pool for class {c}: have {len(pos_in_pool)}, need {n_per_class}")
        chosen = random_state.choice(pos_in_pool, size=n_per_class, replace=False)
        idxs.append(chosen)
    return np.concatenate(idxs)

idx_test = stratified_from_pool(remaining_idx, Y_remaining, n_per_class=test_per_class, random_state=rng)

In [ ]:
X_bg_w2v = X_train_w2v[idx_bg]
X_bg_c2v = X_train_c2v[idx_bg]
Y_bg = Y[idx_bg]

X_test_w2v = X_train_w2v[idx_test]
X_test_c2v = X_train_c2v[idx_test]
Y_test = Y[idx_test]

print("Background shapes:", X_bg_c2v.shape, X_bg_w2v.shape, Y_bg.shape)
print("Test shapes:", X_test_c2v.shape, X_test_w2v.shape, Y_test.shape)

In [ ]:
import shap

In [ ]:
background = [
    X_bg_c2v.astype(np.int32),
    X_bg_w2v.astype(np.float32)
]

testset = [
    X_test_c2v.astype(np.int32),
    X_test_w2v.astype(np.float32)
]

# ---- WRAPPER FUNCTION CHO MODEL ----
def f(inputs):
    # inputs = [batch_char, batch_word]
    return model(inputs, training=False)

print("Tạo DeepExplainer...")
explainer = shap.DeepExplainer(f, background)

print("Tính SHAP...")
shap_values = explainer.shap_values(testset)


In [ ]:
print("Type of shap_values:", type(shap_values))

# Normalize output: handle cases
if isinstance(shap_values, list) and len(shap_values) == 2:
    char_shap = np.array(shap_values[0])
    word_shap = np.array(shap_values[1])
else:
    # If shap_values is single np.array with shape (2, m, ... ) or (m, D)
    sv = np.array(shap_values)
    if sv.ndim == 3 and sv.shape[0] == 1:
        # shape (1, m, D_concat) -> take sv[0]
        sv = sv[0]
    if sv.ndim == 2 and sv.shape[1] == (X_c2v.shape[1] + X_w2v.shape[1]):
        # split
        char_shap = sv[:, :X_c2v.shape[1]]
        word_shap = sv[:, X_c2v.shape[1]:]
    else:
        raise RuntimeError("Unexpected shap_values shape: " + str(sv.shape))

print("char_shap shape:", char_shap.shape)  # expect (n_test, 200)
print("word_shap shape:", word_shap.shape)  # expect (n_test, 50)

In [ ]:
feature_names_word = [f"w_{i}" for i in range(word_dim)]
shap.summary_plot(word_shap, X_test_w2v, feature_names=feature_names_word, show=True)

# For char sequence (200 positions) - positions can be named pos_0..pos_199
feature_names_char = [f"pos_{i}" for i in range(X_c2v.shape[1])]
# But note: these positions are token ids. To map token id -> char you can use tokenizer.index_word (if available)
shap.summary_plot(char_shap, X_test_c2v, feature_names=feature_names_char, show=True)